# 6. Evolve-Filter, time dep, N varies

* $\delta = 1/nx$
* $a = a_D$

In [4]:
from dolfin import *
from __future__ import print_function
import numpy as np
import sympy as sym
import csv

x, y, t = sym.symbols('x[0], x[1], t')
sigma = 1.0
mu = 10**(-5) 
b=as_vector([2.0, 3.0])

# Iliescu Exact Solution
c = 16.0*sym.sin(sym.pi*t)
h = x*(1-x)*y*(1-y)
g = 2*mu**(-0.5)*(0.25**2 - (x - 0.5)**2 - (y - 0.5)**2 )
ue = c*h*(0.5+sym.atan(g)/sym.pi)

# ------------------------------------------ #

ue = sym.simplify(ue)
u_code = sym.printing.ccode(ue)
u_code = u_code.replace('M_PI','DOLFIN_PI')

# du/dt - mu*Laplace(u) + div(bu) + sigma*u = f
fe = sym.diff(ue,t)
fe += - mu*(sym.diff(sym.diff(ue,x),x) + sym.diff(sym.diff(ue,y),y))
fe += b[0]*sym.diff(ue,x) + b[1]*sym.diff(ue,y)
fe += sigma*ue

f_code = sym.printing.ccode(fe)
f_code = f_code.replace('M_PI','DOLFIN_PI')

# print('u_code = ' + u_code + '\n')
# print('f_code = ' + f_code)

def compute_errors(u_e, u, t, mesh):
	L2n = errornorm(u_e, u, norm_type='L2', degree_rise=3, mesh=mesh)
	H1n = errornorm(u_e, u, norm_type='H1', degree_rise=3, mesh=mesh)
	return L2n, H1n

def compute_extrema(u, t):
    maxval = np.amax(u.vector().get_local())
    minval = np.amin(u.vector().get_local())
    return maxval, minval

# filter solving

def deconv_filter(u_, delta, Q, boundary):
    u_tilde = TrialFunction(Q)
    v = TestFunction(Q)

    F_Hfilter = v*u_tilde*dx + delta*delta*dot(grad(v), grad(u_tilde))*dx - v*u_*dx 

    a_Hfilter = lhs(F_Hfilter)
    L_Hfilter = rhs(F_Hfilter)
    
    bc = DirichletBC(Q, u_, boundary)
    
    A_Hfilter = assemble(a_Hfilter)
    bc.apply(A_Hfilter)

    b_Hfilter = assemble(L_Hfilter)
    bc.apply(b_Hfilter)

    solver = LUSolver(A_Hfilter)
    u_tilde = Function(Q)
    solver.solve(u_tilde.vector(), b_Hfilter)
    
    return u_tilde

def EF_t(remark, N, filename, delta, nx, dt, T, u_code, f_code, sigma, mu, velocity, chi):
    degree = 2
    folder = 'FEFfigs/'+remark+"/"

    
    t = 0.0
    u_exact = Expression(u_code, degree = degree+1, t = t)
    f = Expression(f_code, degree = degree+1, t = t)

    mesh = UnitSquareMesh(nx,nx)
    Q = FunctionSpace(mesh, "CG", degree)

    # Set up boundary condition
    u_D = Expression(u_exact.cppcode, degree = degree+1, t = t)
    
    def boundary(x, on_boundary):
        return on_boundary

    
# ------------ STEP 1: Solve on coarse grid ----------------------#

    # Test and trial functions
    u, v = TrialFunction(Q), TestFunction(Q)
#     u_n = Function(Q)
    u_n = interpolate(u_D, Q)
    u_ = Function(Q)

    # Galerkin variational problem
    F = (u - u_n)*v*dx
    F += dt*(mu*dot(grad(v), grad(u))*dx + v*dot(velocity, grad(u))*dx + sigma*v*u*dx - f*v*dx)

    # Create bilinear and linear forms
    a1 = lhs(F)
    L = rhs(F)

    # Assemble matrices
    A1 = assemble(a1)

    # Create progress bar
    progress = Progress('Time-stepping')
    set_log_level(PROGRESS)

    # Outputting files
    #out_file_uexact = File(folder+"evolve_u_exact_"+str(nx)+".pvd") 
    #out_file_ubar = File(folder+str(mu)+"evolve_u_"+str(nx)+".pvd") 
    ue = interpolate(u_exact, Q)
    out_file_ind = File(folder+"EF_aD"+str(nx)+'_N'+str(N)+".pvd")
    out_file_ufilter = File(folder+"EF_u_"+str(nx)+'_N'+str(N)+".pvd") 

    # Save t = 0.0
    #out_file_uexact << (ue, float(t))
    
    
    u_.rename('u','u')
    indicator = Expression('0.0', degree=2, t=0.0)
    indicator = interpolate(indicator, Q)
    indicator.rename('a','a')
    
    #out_file_ufilter << (u_, float(t))
    #out_file_ind << (indicator, float(t))
    
    while t - T + dt < DOLFIN_EPS:
        # Step 1 Solve on Coarse Grid
        t += dt

        u_D.t = t
        f.t = t
        u_exact.t = t
        
        b = assemble(L)
        bc = DirichletBC(Q, u_D, boundary)
        bc.apply(A1)
        bc.apply(b)
        
        solve(A1, u_.vector(), b)#, 'gmres')
        progress.update(t / T)
        
#         if near(t,0.5):
#             errors = compute_errors(u_exact, u_, t, mesh)
#             print("Before Filter \n" + errors)

        #out_file_uexact << (ue, float(t))
        #out_file_ubar << (u_, float(t))
        
# ------------ STEP 2: Helmholtz filter ----------------------#
    
        if N == 0:
            u_tilde0 = deconv_filter(u_, delta, Q, boundary)
            DF = Expression('u_tilde0', degree = degree, u_tilde0 = u_tilde0)

        if N == 1:
            u_tilde0 = deconv_filter(u_, delta, Q, boundary)
            u_tilde1 = deconv_filter(u_tilde0, delta, Q, boundary)
            DF = Expression('2.0*u_tilde0 - u_tilde1', degree = degree, u_tilde0 = u_tilde0, u_tilde1 = u_tilde1)

        if N == 2:
            u_tilde1 = deconv_filter(u_, delta, Q, boundary)
            u_tilde2 = deconv_filter(u_tilde1, delta, Q, boundary)
            u_tilde3 = deconv_filter(u_tilde2, delta, Q, boundary)
            DF = Expression('3.0*u_tilde1 - 3.0*u_tilde2 + u_tilde3', degree = degree, u_tilde1 = u_tilde1, u_tilde2 = u_tilde2, u_tilde3 = u_tilde3)

        if N == 3:
            u_tilde1 = deconv_filter(u_, delta, Q, boundary)
            u_tilde2 = deconv_filter(u_tilde1, delta, Q, boundary)
            u_tilde3 = deconv_filter(u_tilde2, delta, Q, boundary)
            u_tilde4 = deconv_filter(u_tilde3, delta, Q, boundary)
            DF = Expression('3.0*u_tilde1 - 3.0*u_tilde2 + u_tilde3 + u_tilde1 - 3.0*u_tilde2 + 3.0*u_tilde3 - u_tilde4', degree = degree, u_tilde1 = u_tilde1, u_tilde2 = u_tilde2, u_tilde3 = u_tilde3, u_tilde4 = u_tilde4)
            
        # Compute the indicator function N = 0
        indicator = Expression('sqrt((a-b)*(a-b))', degree = 2, a = u_, b = DF)
        indicator = interpolate(indicator, Q)
        max_ind = np.amax(indicator.vector().get_local())

        if max_ind < 1:
            max_ind = 1.0

        indicator = Expression('a/b', degree = 2, a = indicator, b = max_ind)
        indicator = interpolate(indicator, Q)

        indicator.rename('a','a')
        #out_file_ind << (indicator, t)

        # Apply the filter
        u_bar = TrialFunction(Q)
        F_filter = v*u_bar*dx + delta*delta*dot(grad(v), indicator*grad(u_bar))*dx - v*u_*dx 

        a_filter = lhs(F_filter)
        L_filter = rhs(F_filter)

        A_filter = assemble(a_filter)
        bc.apply(A_filter)

        b_filter = assemble(L_filter)
        bc.apply(b_filter)

        solver = LUSolver(A_filter)
        u_bar = Function(Q)
        solver.solve(u_bar.vector(), b_filter)

        u_bar.rename('u','u')
#         out_file_ufilter << (u_bar, float(t))

        
        u_n.assign((1 - chi)*u_+ chi*u_bar)
        
    out_file_ind << (indicator, float(t))
    out_file_ufilter << (u_bar, float(t))   
    L2, H1 = compute_errors(u_exact, u_bar, t, mesh)
    maxval, minval = compute_extrema(u_bar, t)
    
    print(nx,",",L2,",",H1,",",maxval,",",minval)

* 4/23/18: noticed that the error calculated was not the updated solution (u_bar). So fixing that. Directory with just "N_" is the old data.

filename = 'h'
dt = 0.01
T = 0.5

print("nx L2 H1 maxval minval")
remark = "_042318"
N = 3
print("N = "+str(N))
for nx in [25, 50, 100, 200, 400]:

        delta = 1.0/nx
        EF_t(remark, N, filename, delta, nx, dt, T, u_code, f_code, sigma, mu, b)

without GMRES in evolve step

In [19]:
filename = 'h'
dt = 0.001
T = 0.5

print("nx L2 H1 maxval minval")
remark = "_043018"
N = 0
print("N = "+str(N))
for nx in [200]:

        delta = 1.0/nx
        EF_t(remark, N, filename, delta, nx, dt, T, u_code, f_code, sigma, mu, b)

nx L2 H1 maxval minval
N = 0
200 , 0.0111923250778 , 1.99830333613 , 0.99132189307 , -0.0305270047004


added in chi

In [25]:
filename = 'h'
dt = 0.001
T = 0.5
chi = 1.0/256

print("nx L2 H1 maxval minval")
remark = "_043018"
N = 0
print("N = "+str(N))
for nx in [400]:

        delta = 1.0/nx
        EF_t(remark, N, filename, delta, nx, dt, T, u_code, f_code, sigma, mu, b, chi)

nx L2 H1 maxval minval
N = 0
200 , 0.00248643152111 , 2.0605062148 , 1.00053801993 , -0.0114030456319


In [26]:
filename = 'h'
dt = 0.001
T = 0.5
chi = 1.0/256

print("nx L2 H1 maxval minval")
remark = "_043018"
N = 0
print("N = "+str(N))
for nx in [400]:

        delta = 1.0/nx
        EF_t(remark, N, filename, delta, nx, dt, T, u_code, f_code, sigma, mu, b, chi)

nx L2 H1 maxval minval
N = 0
400 , 0.000449197654612 , 0.638915704957 , 0.991713242 , -0.000935873505142


In [5]:
filename = 'h'
dt = 0.001
T = 0.5
chi = 1.0/16
print("nx L2 H1 maxval minval")
remark = "_050118_dt1e-3"
nx = 200
print('nx = ', nx)
for N in [1,2,3]:
    print("N = "+str(N))
    delta = 1.0/nx
    EF_t(remark, N, filename, delta, nx, dt, T, u_code, f_code, sigma, mu, b, chi)

nx L2 H1 maxval minval
nx =  200
N = 1
200 , 0.00238153344296 , 1.87847151622 , 0.998843056643 , -0.0090966410653
N = 2
200 , 0.00226767727685 , 1.89400748661 , 0.998955608772 , -0.00835552922583
N = 3
200 , 0.00220186112282 , 1.90654234181 , 0.999038384746 , -0.00839360953589


In [6]:
filename = 'h'
dt = 0.001
T = 0.5
chi = 1.0/256
print("nx L2 H1 maxval minval")
remark = "_050118_dt1e-3"
nx = 400
print('nx = ', nx)
for N in [1,2,3]:
    print("N = "+str(N))
    delta = 1.0/nx
    EF_t(remark, N, filename, delta, nx, dt, T, u_code, f_code, sigma, mu, b, chi)

nx L2 H1 maxval minval
nx =  400
N = 1
400 , 0.000330887082646 , 0.521152303703 , 0.991714071727 , -0.000935504819185
N = 2
400 , 0.000289179279849 , 0.481799914227 , 0.991714201234 , -0.000935852495608
N = 3
400 , 0.00027141099988 , 0.469704353831 , 0.991714261194 , -0.000936159236114


In [5]:
filename = 'h'
dt = 0.001
T = 0.5
chi = 1.0/4
print("nx L2 H1 maxval minval")
remark = "_050218_dt1e-3"
nx = 100
print('nx = ', nx)
for N in [0,1,2,3]:
    print("N = "+str(N))
    delta = 1.0/nx
    EF_t(remark, N, filename, delta, nx, dt, T, u_code, f_code, sigma, mu, b, chi)

nx L2 H1 maxval minval
nx =  100
N = 0
100 , 0.0181579499162 , 3.16881863663 , 1.00635669358 , -0.0438363908733
N = 1
100 , 0.0163473543077 , 3.26090267714 , 1.01236416924 , -0.0411020518644
N = 2
100 , 0.0151892995012 , 3.28680685018 , 1.01444672961 , -0.0377961896146
N = 3
100 , 0.0144077428195 , 3.29501194498 , 1.01575312558 , -0.0358656565477
